# Traffic data aggregation using network distance metrics and cokriging
Final project for GEOG 8102  
Yue Lin (lin.3326 at osu.edu)  
Created: 4/14/2020

## Mapping points to non-Euclidean space

Read graph data

In [2]:
import csv
with open('d04_tomtom_od.csv', newline='') as csvfile:
    alllines = csv.reader(csvfile)
    graph = []
    for line in alllines:
        item = []
        item.append(int(line[2]))
        item.append(int(line[3]))
        item.append(float(line[5]))
        graph.append(item)
    print(graph[:5])

[[1, 1, 0.0], [1, 220, 3145.43243], [1, 290, 3189.500113], [1, 221, 6100.009007], [1, 328, 10813.73763]]


Costruct distance matrix

In [3]:
## number of nodes
n = max([g[1] for g in graph])

## distance matrix
INF = float(0)
distance_matrix = [ [INF] * n for i in range(n)]
for g in graph:
    i = g[0] - 1
    j = g[1] - 1
    d = g[2]
    distance_matrix[i][j] = d
    distance_matrix[j][i] = d
print("distance matrix:", distance_matrix)

distance matrix: [[0.0, 17150.04903, 38489.02037, 30521.88535, 38910.11525, 47346.56977, 65005.18074, 54945.04164, 45410.74952, 35285.65345, 48192.22321, 58941.03034, 50443.99102, 58864.40456, 50384.4079, 310220.3396, 300966.3949, 291782.5417, 282142.6155, 272426.359, 263021.8159, 253470.5253, 243461.3313, 131598.9736, 85243.99641, 81013.04586, 78640.83424, 98252.23825, 85780.53781, 76490.10952, 83262.51987, 67993.00164, 75941.61896, 85618.51851, 80728.24376, 71665.44172, 62150.74299, 53332.85956, 80735.24844, 71698.46438, 62168.73015, 53358.71919, 86714.67648, 98887.1679, 106076.6163, 0.0, 0.0, 45100.75552, 54182.84812, 63485.98439, 72855.65709, 82000.06407, 91306.4935, 100798.9463, 54877.23971, 49843.44762, 41226.93759, 37185.75625, 50702.88234, 84539.08728, 0.0, 141633.9084, 151005.9799, 161529.4588, 167742.9057, 177192.8017, 186805.5161, 191494.8794, 198046.7354, 207958.6603, 126408.809, 198155.8168, 188929.0457, 180036.351, 170957.7042, 230307.666, 46373.86803, 23296.3639, 14919.2

Construct b matrix

In [4]:
# extract a column
def column(matrix, i):
    return [row[i] for row in matrix]


b_matrix = [ [INF] * n for i in range(n)]

d_sum = 0
for i in range(n):
    for j in range(n):
        d = distance_matrix[i][j]
        d_sum = d_sum + d*d
       
for i in range(n):
    for j in range(n):
        d = distance_matrix[i][j]
        
        d_x = distance_matrix[i]
        d_x_sum = 0
        for k in range(n):
            d = distance_matrix[i][k]
            d_x_sum = d_x_sum + d*d

        d_y = column(distance_matrix, j)
        d_y_sum = 0
        for k in range(n):
            d = distance_matrix[k][j]
            d_y_sum = d_y_sum + d*d
        
        d = distance_matrix[i][j]
        
        b_matrix[i][j] = 1/2 * (-d*d + 1/n * d_x_sum + 1/n * d_y_sum - 1/(n*n) * d_sum)

print("b matrix:", b_matrix[:5])

b matrix: [[1549401422.6990204, 940385610.4724827, 1601051355.1980515, 1126574062.5139103, 1618509719.731535, 2111851340.2454815, -860865591.1946735, -239411402.76304436, 311676622.756979, 854079939.5269375, -55799392.65058899, -863570316.2862816, -401633624.3778858, -859839661.1441345, -398781746.9592762, -9371931878.952692, -9042110270.179987, -8718432647.155054, -8382588107.549658, -8048133677.581137, -7728280852.036432, -7407335330.737459, -7075218671.362196, -2824087743.3923655, 4086632882.277214, 3946129531.995899, 3887833048.4313984, 4583093960.933805, 4103266192.5235806, 3792992783.0895786, 4077752829.441923, 3442706800.9847813, 3738472406.4286747, 4099159847.382971, 4002362610.6808186, 3639315106.1976166, 3261984134.1889133, 2933179549.570799, 4002637032.043165, 3640675680.0455093, 3262676492.2631435, 2934140955.92502, 4213068066.2990265, 4588239550.366941, 4846303147.944254, -4410143806.475471, -4408620028.412811, 3338883955.916855, 3865744501.6017857, 4401745305.53248, 49378

Get the new point set

In [9]:
import numpy as np
import scipy.linalg as la

B = np.array(b_matrix)
eigvals, eigvecs = la.eig(B)
sorted_eigvals = sorted(eigvals, reverse = True)

sorted_eigvals = np.array(sorted_eigvals[:2])
V = np.diag(sorted_eigvals**(0.5))
print(V.shape)

eigvals = eigvals.tolist()
index1 = eigvals.index(sorted_eigvals[0])
Q1 = np.array([eigvecs[index1]])
index2 = eigvals.index(sorted_eigvals[1])
Q2 = np.array([eigvecs[index2]])
Q = np.concatenate((Q1, Q2), axis=0)
print(Q.shape)

S_new = np.dot(np.transpose(Q), V)
print(S_new.shape)
print(S_new[0][0])

(2, 2)
(2, 418)
(418, 2)
(36736.91777467165+0j)


Write new coordinates to file

In [10]:
import csv
 
default_text = 'Some Text'
with open('d04_tomtom.csv', 'r') as read_obj, \
        open('d04_tomtom_1.csv', 'w', newline='') as write_obj:
    csv_reader = csv.reader(read_obj)
    csv_writer = csv.writer(write_obj)
    
    i = 0
    for row in csv_reader:
        row.append(S_new[i][0])
        row.append(S_new[i][1])
        i = i + 1
        
        csv_writer.writerow(row)